from https://colab.research.google.com/drive/115ba3EFCT0PvyXzFNv9E18QnKiyyjsm5?usp=sharing

In [1]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline, BitsAndBytesConfig
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
import torch
import ipywidgets as widgets

In [2]:
tokenizer = LlamaTokenizer.from_pretrained("chavinlo/alpaca-native")
base_model = LlamaForCausalLM.from_pretrained("chavinlo/alpaca-native", 
                                              device_map='auto', 
                                              load_in_8bit=True, 
                                              offload_folder='./offload/')

/Users/skelley/anaconda3/envs/mytorch/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /Users/skelley/anaconda3/envs/mytorch/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so
'NoneType' object has no attribute 'cadam32bit_grad_fp32'
CUDA SETUP: Loading binary /Users/skelley/anaconda3/envs/mytorch/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so...
dlopen(/Users/skelley/anaconda3/envs/mytorch/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so, 0x0006): tried: '/Users/skelley/anaconda3/envs/mytorch/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so' (not a mach-o file), '/System/Volumes/Preboot/Cryptexes/OS/Users/skelley/anaconda3/envs/mytorch/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so' (no such file), '/Users/skelley/a

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
pipe = pipeline(
    "text-generation",
    model=base_model, 
    tokenizer=tokenizer, 
    max_length=256,
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.2
)
local_llm = HuggingFacePipeline(pipeline=pipe)
template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: 
{instruction}

Answer:"""
prompt = PromptTemplate(template=template, input_variables=["instruction"])
llm_chain = LLMChain(prompt=prompt, llm=local_llm)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [4]:
question = "What is the capital of England?"
print(llm_chain.run(question))

/Users/skelley/anaconda3/envs/mytorch/lib/python3.10/site-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 question = "What is the capital of England?"                                                 │
│ ❱ 2 print(llm_chain.run(question))                                                               │
│   3                                                                                              │
│                                                                                                  │
│ /Users/skelley/anaconda3/envs/mytorch/lib/python3.10/site-packages/langchain/chains/base.py:290  │
│ in run                                                                                           │
│                                                                                                  │
│   287 │   │   if args and not kwargs:                                                            │
│   288 │   │   │   if len(args) != 1:                                                             │
│   289 │   │   │   │   raise ValueError("`run` supports only one positional argument.")           │
│ ❱ 290 │   │   │   return self(args[0], callbacks=callbacks, tags=tags)[_output_key]              │
│   291 │   │                                                                                      │
│   292 │   │   if kwargs and not args:                                                            │
│   293 │   │   │   return self(kwargs, callbacks=callbacks, tags=tags)[_output_key]               │
│                                                                                                  │
│ /Users/skelley/anaconda3/envs/mytorch/lib/python3.10/site-packages/langchain/chains/base.py:166  │
│ in __call__                                                                                      │
│                                                                                                  │
│   163 │   │   │   )                                                                              │
│   164 │   │   except (KeyboardInterrupt, Exception) as e:                                        │
│   165 │   │   │   run_manager.on_chain_error(e)                                                  │
│ ❱ 166 │   │   │   raise e                                                                        │
│   167 │   │   run_manager.on_chain_end(outputs)                                                  │
│   168 │   │   final_outputs: Dict[str, Any] = self.prep_outputs(                                 │
│   169 │   │   │   inputs, outputs, return_only_outputs                                           │
│                                                                                                  │
│ /Users/skelley/anaconda3/envs/mytorch/lib/python3.10/site-packages/langchain/chains/base.py:160  │
│ in __call__                                                                                      │
│                                                                                                  │
│   157 │   │   )                                                                                  │
│   158 │   │   try:                                                                               │
│   159 │   │   │   outputs = (                                                                    │
│ ❱ 160 │   │   │   │   self._call(inputs, run_manager=run_manager)                                │
│   161 │   │   │   │   if new_arg_supported                                                       │
│   162 │   │   │   │   else self._call(inputs)                                                    │
│   163 │   │   │   )                                                                              │
│                                                            

In [ ]:
def on_button_clicked(b):
    with output:
        print(llm_chain.run(question))
output = widgets.Output()
question = widgets.Text()
button = widgets.Button(description='Chat')
button.on_click(on_button_clicked)
widgets.VBox([question, button, output])

In [ ]:
import torch
# check if MPS is available:
torch.backends.mps.is_available()

In [ ]:
print(to